---

来自 https://github.com/klyshko/md_python/blob/master/Lecture.ipynb

---

主要学习：

- 通过numpy掌握基本的模拟思想
- 利用openmm模拟蛋白轨迹

## 1.牛顿运动定律
牛顿第二定律的公式为 $m \frac{d^{2}r(t)}{dt^2} = F = -\nabla U(r)$，具体可以分解为以下两个公式:
$$
\frac{dr(t)}{dt} = v(t)\\
m\frac{dv(t)}{dt} = F(t)
$$
具体可以模拟为:
$$
v_{n+1} = v_n + \frac{F_n}{m}dt\\
r_{n+1} = r_n + v_{n+1}dt
$$

### 1.1 模拟自由落体运动
>We want to know the dynamics of a green apple (m=0.3 kg) tossed horizontally at 10 cm/s speed from the top of the Toronto CN Tower (553 m) for the first 10 seconds.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

# Setup the figure and axes...
fig, ax = plt.subplots(figsize=(8,8))

## Adjust axes limits according to your problem. Here we don't need more than a couple of meters left or right, and 600 meters up
ax.set(xlim=(-2, 2), ylim=(0, 600), xlabel='Position, meters', ylabel='Height, meters', title='Apple falling from CN tower')

# parameters of the problem
T = 10. #s
m = 0.3 #kg
g = 9.8 #m/s^2
v0x = -0.1 #m/s
H = 553. #m

# setting a timestep to be 50 ms
dt = 0.05 #s
N = int(T / dt)

# Allocating arrays for 2D problem
v = np.zeros((N+1, 2))
r = np.zeros((N+1, 2))
f = np.zeros((N+1, 2))

# initial conditions:
r[0] = np.array([0., H])
v[0] = np.array([-v0x, 0.])

# the only force is gravity
f[:] = np.array([0., -m * g])

## Run dynamics:
for n in range(N):
    v[n+1] = v[n] + f[n]/m * dt
    r[n+1] = r[n] + v[n+1] * dt

## drawing the first data point  
scat = ax.scatter(r[0,0], r[0,1], marker='o', c='g', s=200)

## animating 
def animate(i):
    scat.set_offsets(r[i])

ani = animation.FuncAnimation(fig, func=animate, frames=N)
## this function will create a lot of *.png files in a folder 'CNtower_frames'
## and create an HTML page with a simulation
ani.save('CNtower.html', writer=animation.HTMLWriter(fps= 1//dt))
plt.close()
#ani.save('CNtower.mp4', fps= 1//dt)

In [2]:
from IPython.display import HTML
HTML('CNtower.html')


当一个封闭系统的颗粒通过对势能来发生相互作用时，作用在每一个颗粒i上的力取绝于它和所有其他颗粒j的相互作用
$$
m_{i}\frac{d^2r_i(t)}{dt^2}=\sum\limits_{j}F_{ij}(t)=-\sum\limits_{j}\nabla_{i}U(|r_{ij}(t)|)
$$
$r_{ij}=\sqrt{(x_i-x_j)^2+(y_i-y_j)^2+(z_i-z_j)^2}$是i,j物体的相对距离，对于$i,j\in (1,N)$

### 1.2利用胡可定律模拟三体问题(three body problem)
Hooke's Law (pairwise potential)
$$
\begin{aligned}
U(r_{ij}) &= \frac{K_s}{2}(r_{ij}-r_0)^2\\
F_{ij} &= \nabla_i U(r_{ij}) = -K_s (r_ij-r_0)\nabla_i r_{ij} 
\end{aligned}
$$
>Problem : We want to know the dynamics of 3 particles m=1 kg connected to each other with invisible springs with Ks=5N/m, and r0=1m initially located at (0, 2), (2, 0) and (-1, 0) on the 2D plane for the first 10 seconds of their motion.

>While a system of 3 bodies interacting gravitationally is $\bold{chaotic}$, a system of 3 bodies interacting elastically isn't.

In [4]:
# Setup the figure and axes...
fig, ax = plt.subplots(figsize=(6,6))
ax.set(xlim=(-3.5, 3.5), ylim=(-3.5, 3.5), ylabel='meters', xlabel='meters', title='3-Body problem')

# parameters of the problem
T = 10. #s
m = 1.0 #kg
ks = 5 #N/m
r0 = 1. #m

# setting a timestep to be 50 ms
dt = 0.05 #s
N = int(T / dt)

# Allocating arrays for 2D problem: first axis - time. second axis - particle's number. third - coordinate
v = np.zeros((N+1, 3, 2))
r = np.zeros((N+1, 3, 2))
f = np.zeros((N+1, 3, 2))

# initial conditions for 3 particles:
r[0,0] = np.array([0., 2.])
r[0,1] = np.array([2., 0.])
r[0,2] = np.array([-1., 0.])

def compute_forces(n):
    '''The function computes forces on each pearticle at time step n'''
    for i in range(3):
        for j in range(3):
            if i != j:
                rij = r[n,i] - r[n,j]
                rij_abs = np.linalg.norm(rij)
                f[n, i] -= ks * (rij_abs - r0) * rij / rij_abs 
## Run dynamics:
for n in range(N):
    compute_forces(n)
    v[n+1] = v[n] + f[n]/m * dt
    r[n+1] = r[n] + v[n+1] * dt

## drawing and animating 
scat = ax.scatter(r[0,:,0], r[0,:,1], marker='o', c=['b', 'k', 'r'], s=1000)

def animate(i):
    scat.set_offsets(r[i])

ani = animation.FuncAnimation(fig, animate, frames=N)
plt.close()
## this function will create a lot of *.png files in a folder '3Body_frames'
ani.save('3body.html', writer=animation.HTMLWriter(fps= 1//dt))

In [5]:
HTML('3body.html')

simulation的基本原理，先确定相互作用力的形式（根据势能求梯度而来），再确定每一帧的速度、位置，重复循环
$$
\begin{aligned}
v_{t+1}&=v_{t}+\frac{f(t)}{m} \Delta t\\
r_{t+1}&=r_{t}+v_{t+1}\Delta t
\end{aligned}
$$

## 2.蛋白质运动模拟

蛋白质可以视为成千上万个粒子所组成的负责系统，彼此之间通过复杂的能量函数连接，整体的能量可以分解为共价键能量$V_b$和非共价键能量$V_{nb}$，也就是
$$
V = V_b + V_{nb}
$$
共价键势能主要包含以下四项，基本上都是harmonic potential，除了二面角是正弦函数能量
- harmonic bond part
- harmonic angle 
- two types of torsion (dihedral) angles: proper and improper
$$
V_b = \sum\limits_{bonds}\frac{1}{2}K_b(b-b_0)^2 + \sum\limits_{angles}K_{\theta}(\theta-\theta_0)^2+K_{\phi}(1-cos(n\phi-\phi_{0}))+\sum\limits_{impropers}K_{\psi}(\psi-\psi_0)^2
$$
非共价势能主要包含静电力和LJ势能(范德华力)
$$
V_{nb} = \sum\limits_{ij}\left(\frac{q_iq_j}{4\pi\epsilon_0\epsilon r_{ij}} + \epsilon_{ij} \left[\left(\frac{\sigma^{min}_{ij}}{r_{ij}}\right)^{12}-2\left(\frac{\sigma^{min}_{ij}}{r_{ij}}\right)^6\right]\right)
$$
$r_ij$表示相互作用两个原子的距离，$q_i$和$q_j$表示个字的电荷，

In [2]:
from openmm.app import *
from openmm import *
from simtk.unit import *
import MDAnalysis as md
import py3Dmol
from sys import stdout